In [1]:
import pandas as pd
import datetime
#need to update this to remove Source mapping

In [2]:
fakeasset_df = pd.read_csv("01-tw-fakeassets.csv")

# Incorporating rdf transformation

In [3]:
#define key namespaces to be used in the mapping
famo = "http://ontology.eil.utoronto.ca/FAMO/famo/"
ex = "http://example.com/data/"
rdf = "http://www.w3.org/1999/02/22-rdf-syntax-ns#"
owl = "http://www.w3.org/2002/07/owl#"

#current datetime to be used for identifier suffix
dt_now = datetime.datetime.now().strftime("%d%b%Y_%H%M%S")

## Asset mapping

* **TODO** test for null values in all columns before mapping
* **Note** for the purposes of this mapping, the role records are used to generate corresponding DH entities. Each Role record is represented as pinned to the corresponding DH entity (to instantiate the data set), and initialized with the wms entity number as canonical.

In [4]:
ttl_list = []
#prefixes
ttl_list.append(f"PREFIX famo: <{famo}>"+'\n')
ttl_list.append(f"PREFIX ex: <{ex}>"+'\n')
ttl_list.append(f"PREFIX rdf: <{rdf}>"+'\n')
ttl_list.append(f"PREFIX owl: <{owl}>"+'\n')

for i in range(0,len(fakeasset_df)):
    #record instantiation and interpretation
    ttl_list.append(f"ex:wms_assetrec_{fakeasset_df['Asset'][i]} rdf:type famo:Record."+'\n')
    #instantiated in wms_datasystem
    ttl_list.append(f"ex:wms_assetrec_{fakeasset_df['Asset'][i]} famo:instantiatedIn ex:wms_datasystem."+'\n')
    ttl_list.append(f"ex:wms_assetrec_{fakeasset_df['Asset'][i]} famo:hasInterpretation ex:wms_assetinterpretation_{fakeasset_df['Asset'][i]}."+'\n')
    #denotes corresponding dhid
    ttl_list.append(f"ex:wms_assetinterpretation_{fakeasset_df['Asset'][i]} famo:denotes ex:dhid_{fakeasset_df['Asset'][i]}."+'\n')
    ttl_list.append(f"ex:dhid_{fakeasset_df['Asset'][i]} rdf:type famo:Asset."+'\n')
    #assert class type for dhid
        #note: this isn't part of the semantic mapping of the wms data but a means of initializing the dh entities
    ttl_list.append(f"ex:dhid_{fakeasset_df['Asset'][i]} rdf:type <{fakeasset_df['AssetClass'][i]}>."+'\n')
    
    #record properties
    #asset id
    ttl_list.append(f"ex:wms_assetrec_{fakeasset_df['Asset'][i]} famo:representsSubjectOf ex:wms_assetnum_{fakeasset_df['Asset'][i]}_{dt_now}."+'\n')
    ttl_list.append(f"ex:wms_assetnum_{fakeasset_df['Asset'][i]}_{dt_now} a famo:IdentificationNumberProperty."+'\n')
    ttl_list.append(f"ex:wms_assetnum_{fakeasset_df['Asset'][i]}_{dt_now} famo:hasValue '{fakeasset_df['AssetNum'][i]}'."+'\n')

    #asset serial number
    ttl_list.append(f"ex:wms_assetrec_{fakeasset_df['Asset'][i]} famo:representsSubjectOf ex:wms_serialnum_{fakeasset_df['Asset'][i]}_{dt_now}."+'\n')
    ttl_list.append(f"ex:wms_serialnum_{fakeasset_df['Asset'][i]}_{dt_now} a famo:SerialNumberProperty."+'\n')
    ttl_list.append(f"ex:wms_serialnum_{fakeasset_df['Asset'][i]}_{dt_now} famo:hasValue '{fakeasset_df['SerialNum'][i]}'."+'\n')
    
    #asset class
    ttl_list.append(f"ex:wms_assetrec_{fakeasset_df['Asset'][i]} famo:representsSubjectOf ex:wms_assetclass_{fakeasset_df['Asset'][i]}_{dt_now}."+'\n')
    ttl_list.append(f"ex:wms_assetclass_{fakeasset_df['Asset'][i]}_{dt_now} a famo:ClassMembershipRelation."+'\n')
    ttl_list.append(f"ex:wms_assetclass_{fakeasset_df['Asset'][i]}_{dt_now} famo:hasObject <{fakeasset_df['AssetClass'][i]}>."+'\n')

    #status
    ttl_list.append(f"ex:wms_assetrec_{fakeasset_df['Asset'][i]} famo:representsSubjectOf ex:wms_assetstatus_{fakeasset_df['Asset'][i]}_{dt_now}."+'\n')
    ttl_list.append(f"ex:wms_assetstatus_{fakeasset_df['Asset'][i]}_{dt_now} a famo:StatusRelation."+'\n')
    ttl_list.append(f"ex:wms_assetstatus_{fakeasset_df['Asset'][i]}_{dt_now} famo:hasObject <{fakeasset_df['Status'][i]}>."+'\n')

    #asset role
    ttl_list.append(f"ex:wms_assetrec_{fakeasset_df['Asset'][i]} famo:representsObjectOf ex:wms_assetrole_{fakeasset_df['Asset'][i]}_{dt_now}."+'\n')
    ttl_list.append(f"ex:wms_assetrole_{fakeasset_df['Asset'][i]}_{dt_now} a famo:RoleOccupationRelation."+'\n')
    ttl_list.append(f"ex:wms_assetrole_{fakeasset_df['Asset'][i]}_{dt_now} famo:subjectIsRepresentedBy ex:wms_rolerec_{fakeasset_df['ServingRole'][i]}."+'\n')

    #motor size, if applicable
    if (pd.notna(fakeasset_df['MotorSize'][i])):
        ttl_list.append(f"ex:wms_assetrec_{fakeasset_df['Asset'][i]} famo:representsSubjectOf ex:wms_motorsize_{fakeasset_df['Asset'][i]}_{dt_now}."+'\n')
        ttl_list.append(f"ex:wms_motorsize_{fakeasset_df['Asset'][i]}_{dt_now} a famo:MotorsizeProperty."+'\n')
        ttl_list.append(f"ex:wms_motorsize_{fakeasset_df['Asset'][i]}_{dt_now} famo:hasValue '{fakeasset_df['MotorSize'][i]}'."+'\n')

In [5]:
ttl_results = open("01_fakeasset_out.ttl", "a")
#for i in range(0,len(fakeasset_df)-1):
#record instantiation and interpretation
ttl_results.writelines(ttl_list)

ttl_results.close()